In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
home= '/Users/kathleenkanaley/Desktop/grapes_in_space/'

In [16]:
# Batch process

orig_imgs = glob.glob(home +'images_work/images_analysis_ready/*/SkySat/2020*.tif')

In [17]:
print(orig_imgs)
print(len(orig_imgs))

['/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200710_155028_ssc3d2_0012_ar.tif', '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200812_153924_ssc3d2_0013_ar.tif', '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200625_154704_ssc12d2_0013_ar.tif']
3


In [22]:
def get_norm_img(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization is performed using the numpy.linalg.norm() function.
    The function calculates the Frobenius norm of each pixel in the input image
    and divides the pixel by the norm to return a unit vector.
    
    Input:
    img_paths - list of abs. paths to input images
    output_dir - directory to store normalized images
    
    '''
    for i in img_paths:
        with rio.open(i) as src:
            as_array = src.read()
            normalized = as_array/np.linalg.norm(as_array)
           
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =4)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_'+str(os.path.basename(i)), 'w', **kwargs) as dst:
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))

In [1]:
def get_norm_img_8band(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization is performed using the numpy.linalg.norm() function.
    The function calculates the Frobenius norm of each pixel in the input image
    and divides the pixel by the norm to return a unit vector.
    
    Input:
    img_paths - list of abs. paths to input images
    output_dir - directory to store normalized images
    
    '''
    for i in img_paths:
        with rio.open(i) as src:
            as_array = src.read()
            normalized = as_array/np.linalg.norm(as_array)
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =8)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_'+str(os.path.basename(i)), 'w', **kwargs) as dst:
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))
                dst.write_band(5, normalized[4].astype(rio.float32))
                dst.write_band(6, normalized[5].astype(rio.float32))
                dst.write_band(7, normalized[6].astype(rio.float32))
                dst.write_band(8, normalized[7].astype(rio.float32))

In [23]:
#get_norm_img(orig_imgs)